In [0]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, StringType, IntegerType
from pyspark.sql import SparkSession
 
# Define the schema of incoming messages
schema = StructType().add("Time", StringType()).add("V1", StringType()) \
    .add("V2", StringType()) \
    .add("V3", StringType()) \
    .add("V4", StringType()) \
    .add("V5", StringType()) \
    .add("V6", StringType()) \
    .add("V7", StringType()) \
    .add("V8", StringType()) \
    .add("V9", StringType()) \
    .add("V10", StringType()) \
    .add("V11", StringType()) \
    .add("V12", StringType()) \
    .add("V13", StringType()) \
    .add("V14", StringType()) \
    .add("V15", StringType()) \
    .add("V16", StringType()) \
    .add("V17", StringType()) \
    .add("V18", StringType()) \
    .add("V19", StringType()) \
    .add("V20", StringType()) \
    .add("V21", StringType()) \
    .add("V22", StringType()) \
    .add("V23", StringType()) \
    .add("V24", StringType()) \
    .add("V25", StringType()) \
    .add("V26", StringType()) \
    .add("V27", StringType()) \
    .add("V28", StringType()) \
    .add("Amount", StringType()) \
    .add("Class", StringType())
 
spark = SparkSession.builder.appName("KafkaReadExample").getOrCreate()
 
# Define Kafka source options including Confluent Cloud security settings
kafkaOptions = {
    "kafka.bootstrap.servers": "pkc-ld537.ca-central-1.aws.confluent.cloud:9092", 
    "subscribe": "CC_transactions",
    "startingOffsets": "earliest",  
    "kafka.sasl.jaas.config": f'kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username="UDOILAXDAYE4SYBY" password="my_secret_key";',
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.ssl.endpoint.identification.algorithm": "https"
}
 
 #Read from Kafka topic
df = spark \
    .readStream \
    .format("kafka") \
    .options(**kafkaOptions) \
    .load()
 
parsed_df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").select(
    col("key"),
    from_json(col("value"), schema).alias("data")
).select("key", "data.*")
 
# query = parsed_df \
#     .writeStream \
#     .queryName("my_streaming_view") \
#     .outputMode("append") \
#     .format("memory") \
#     .start()

In [0]:
casted_df = parsed_df.select(
    col("V1").cast(DoubleType()),
    col("V2").cast(DoubleType()),
    col("V3").cast(DoubleType()),
    col("V4").cast(DoubleType()),
    col("V5").cast(DoubleType()),
    col("V6").cast(DoubleType()),
    col("V7").cast(DoubleType()),
    col("V8").cast(DoubleType()),
    col("V9").cast(DoubleType()),
    col("V10").cast(DoubleType()),
    col("V11").cast(DoubleType()),
    col("V12").cast(DoubleType()),
    col("V13").cast(DoubleType()),
    col("V14").cast(DoubleType()),
    col("V15").cast(DoubleType()),
    col("V16").cast(DoubleType()),
    col("V17").cast(DoubleType()),
    col("V18").cast(DoubleType()),
    col("V19").cast(DoubleType()),
    col("V20").cast(DoubleType()),
    col("V21").cast(DoubleType()),
    col("V22").cast(DoubleType()),
    col("V23").cast(DoubleType()),
    col("V24").cast(DoubleType()),
    col("V25").cast(DoubleType()),
    col("V26").cast(DoubleType()),
    col("V27").cast(DoubleType()),
    col("V28").cast(DoubleType()),
    col("Amount").cast(DoubleType()),
)

# query = casted_df \
#     .writeStream \
#     .queryName("my_streaming_view") \
#     .outputMode("append") \
#     .format("memory") \
#     .start()

In [0]:
#Example of data
spark.sql("SELECT * FROM my_streaming_view LIMIT 10").show()

+----+-----------------+-----------------+----------------+-----------------+------------------+-------------------+-------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|               V1|               V2|              V3|               V4|                V5|                 V6|                 V7|                V8|                V9|               V10|              V11|              V12|              V13|               V14|              V15|              V16|               V17|               V18|               V19|                V20|               V21|               V22|              V23|               V24

In [0]:
#Load model
from pyspark.ml.classification import RandomForestClassificationModel

model_path = "dbfs:/FileStore/models/rf_model_best"
model = RandomForestClassificationModel.load(model_path)

In [0]:
#Create vector assembler
from pyspark.ml.feature import VectorAssembler, StringIndexer
feature_list = ['V1', 'V2', 'V3', 'V4' ,'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',  'Amount']
assembler = VectorAssembler(inputCols=feature_list, outputCol='features_for_model')


In [0]:
def process_batch(batch_df, batch_id):
    # Apply model to the batch_df
    df = assembler.transform(batch_df)
    predictions = model.transform(df)
    
    # Show predictions
    predictions.select('prediction', 'probability').show()

query = casted_df.writeStream.foreachBatch(process_batch).start()
query.awaitTermination()

+----------+--------------------+
|prediction|         probability|
+----------+--------------------+
|       0.0|[0.96298331796583...|
|       0.0|[0.94332618206735...|
+----------+--------------------+

+----------+--------------------+
|prediction|         probability|
+----------+--------------------+
|       0.0|[0.95638733360261...|
+----------+--------------------+

+----------+--------------------+
|prediction|         probability|
+----------+--------------------+
|       0.0|[0.96298331796583...|
+----------+--------------------+

+----------+--------------------+
|prediction|         probability|
+----------+--------------------+
|       0.0|[0.90093887994208...|
|       0.0|[0.96298331796583...|
|       0.0|[0.95332821764766...|
|       0.0|[0.96298331796583...|
|       0.0|[0.90980112439659...|
|       0.0|[0.95771085797341...|
|       0.0|[0.96298331796583...|
+----------+--------------------+

+----------+--------------------+
|prediction|         probability|
+---------